<a href="https://colab.research.google.com/github/Jeisha/FYP-Temp/blob/main/PART_3_Number_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('drive/MyDrive/FYP')

Mounted at /content/drive


In [ ]:
!pip install db-sqlite3
!pip install easyocr

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 171 kB 5.5 MB/s 
  Created wheel for db-sqlite3: filename=db_sqlite3-0.0.1-py3-none-any.whl size=1794 sha256=a8d8d904991ddd0f0cc9f02cb6cdcc5c88f8944a6c27d251454392dd01adda0b
  Stored in directory: /root/.cache/pip/wheels/02/38/d5/2f54461050571bf5330fee2a37ab1c9b5e7540b0572f1acdab
  Created wheel for db: filename=db-0.1.1-py3-none-any.whl size=3895 sha256=5e0ea67788ec098c755f2a800e3eda7950f9cb05061e7c44714eed48884f1556
  Stored in directory: /root/.cache/pip/wheels/8e/97/82/741d2b360507411ec233d0280d7371faa94b03bde834e4a9be
  Created wheel for antiorm: filename=antiorm-1.2.1-py3-none-any.whl size=31679 sha256=89ecf62e6b00a3c60b55cf3503ca8f1da44103f9d682273b06b3adf47976d7a9
  Stored in directory: /root/.cache/pip/wheels/c5/43/70/e9729370cfff40c49d3e3d05377d54b3ecd71f64e62341ea80
Successfully built db-sqlite3 db antiorm
Looking in indexes: https://pypi

In [ ]:
import easyocr
import glob
import os
import cv2
import pandas as pd
import numpy as np
import sqlite3
from scipy.ndimage import interpolation as inter
from matplotlib import pyplot as plt
from datetime import datetime

paths = {
    'IMAGE_WORK_PATH': os.path.join('image_work'),
    'PLATE_DETECTED_PATH':os.path.join('image_work','PLATE_DETECTED'),
    'OCR_DETECTION_PATH':os.path.join('image_work','OCR_DETECTION'),
    'OCR_DETECTED_PATH':os.path.join('image_work','OCR_DETECTED'),
    'OCR_NOT_DETECTED_PATH':os.path.join('image_work','OCR_NOT_DETECTED')
 }

In [ ]:
def get_img(img):
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    thresh, im_bw = cv2.threshold(gray_img, 150, 100, cv2.THRESH_BINARY)
    def correct_skew(image,image2, delta=1, limit=20):
        def determine_score(arr, angle):
            data = inter.rotate(arr, angle, reshape=False, order=0)
            histogram = np.sum(data, axis=1, dtype=float)
            score = np.sum((histogram[1:] - histogram[:-1]) ** 2, dtype=float)
            return histogram, score

        scores = []
        angles = np.arange(-limit, limit + delta, delta)
        for angle in angles:
            histogram, score = determine_score(image, angle)
            scores.append(score)

        best_angle = angles[scores.index(max(scores))]

        (h, w) = image.shape[:2]
        center = (w // 2, h // 2)
        M = cv2.getRotationMatrix2D(center, best_angle, 1.0)
        corrected = cv2.warpAffine(image2, M, (w, h), flags=cv2.INTER_CUBIC, \
                borderMode=cv2.BORDER_REPLICATE)

        return best_angle, corrected
    return  correct_skew(im_bw,img)

def getAllResult(results):
    result = []
    for i in results:
        number = ''.join(e for e in i[1] if e.isalnum())
        result.append(number)
    number = ''
    for numbers in result:
        number += numbers
    return(number.upper())
        

In [ ]:
db_path = os.path.join('Database','Database.db')
def get_registered_platenum():
  mydb = sqlite3.connect(db_path)
  cmd = "SELECT * FROM registered"
  df = pd.read_sql_query(cmd, mydb)
  return list(df.plate)

def insert_platenum_loggate(timestamp,platenum):
  mydb = sqlite3.connect(db_path)
  cmd = "INSERT INTO loggate (timestamp,plate) VALUES ('{}','{}')".format(timestamp,platenum)
  mydb.execute(cmd)
  mydb.commit()
  print(f"Record logged: {platenum}.")

def check_latest_loggate(platenum):
  mydb = sqlite3.connect(db_path)
  cmd = "SELECT * from loggate order by rowid desc limit 1"
  plate = list((pd.read_sql_query(cmd, mydb)).plate)
  if plate == platenum:
    return False
  else:
    return True

In [ ]:
reader = easyocr.Reader(['en'],gpu=True)
count = 0
start = None
gate = False

while True:
  plate_path = []
  for i in sorted(glob.glob(os.path.join(paths['PLATE_DETECTED_PATH'],'*.png'))):
      plate_path.append(i)

  for i in plate_path:
      image = cv2.imread(i)
      try:
        angle, correction = get_img(image)
        correction_np = np.array(correction)

        # f = plt.figure()
        # f.add_subplot(1,2, 1)
        # plt.imshow(image)
        # f.add_subplot(1,2, 2)
        # plt.imshow(correction)
        # plt.show(block=True)
        
        resultbf = reader.readtext(image)
        resultbf = getAllResult(resultbf)
        resultaf = reader.readtext(correction)
        resultaf = getAllResult(resultaf)

        detected = False
        curimg = image.copy()
        if resultaf != '' or resultbf != '':
            plate_num_list = get_registered_platenum()
            if resultaf in plate_num_list or resultbf in plate_num_list:
                detected = True
                if resultaf in plate_num_list:
                    insert_platenum_loggate(datetime.now(),resultaf)
                    curimg =  correction.copy()
                elif resultbf in plate_num_list:
                    insert_platenum_loggate(datetime.now(),resultbf)
                    curimg =  image.copy()
                    
                img_item = os.path.join(paths['OCR_DETECTED_PATH'],f'{count} - {resultbf} or {resultaf}.png')
                cv2.imwrite(img_item,curimg)
            else:
                img_item = os.path.join(paths['OCR_DETECTION_PATH'],f'{count} - {resultbf} or {resultaf}.png')
                cv2.imwrite(img_item,curimg)
                print(f'Number plate {resultaf} or {resultbf} is not in database.')
        else:
            img_item = os.path.join(paths['OCR_NOT_DETECTED_PATH'],f'{count}.png')
            cv2.imwrite(img_item,curimg)
        os.remove(i)
        count +=1
      except:
        print(f'Error detected, break. Next Loop')
        break

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% CompleteNumber plate JOG2800 or JOG2800 is not in database.
Number plate JOG2800 or JOC2800 is not in database.
Number plate JOG2800 or JOC52800 is not in database.
Number plate JQG2800 or JOG2800 is not in database.
Number plate 62800 or 82800 is not in database.
Record logged: BNF3763.
Record logged: BNF3763.
Record logged: BNF3763.
Record logged: BNF3763.
Number plate 0 or 63 is not in database.
Number plate PJV1301 or PJVI3NR is not in database.
Number plate PJVI30C or PJVIOOC is not in database.
Record logged: PJV1300.
Record logged: PJV1300.
Number plate PUV1300 or PUV1300 is not in database.
Number plate UKHI2112 or UKM2I2 is not in database.
Number plate UKM2112 or UKH2112 is not in database.
Number plate UKM2112 or UKH2112 is not in database.
Number plate UKH2112 or UKM2112 is not in database.
Number plate UKM2112 or UKM2112 is not in database.
Number plate UKM2112 or UKM2112 is not in database.
Number plate

KeyboardInterrupt: ignored